In [1]:
# Importing the libraries
import urllib.request
import zipfile

In [2]:
# Details
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip'
name = 'data.zip'
train_dir = 'data/train/'
urllib.request.urlretrieve(url, name)

('data.zip', <http.client.HTTPMessage at 0x1049a5430>)

In [3]:
# Unzipping
zipped = zipfile.ZipFile(name, 'r')
zipped.extractall(train_dir)
zipped.close()

In [4]:
# Image generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Scale by 1/255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow from directory
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),
    class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [6]:
# CNN architecture
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = tf.keras.models.Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

2022-05-30 00:21:45.748768: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-30 00:21:45.749530: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [9]:
# Summary
# 1.7 mil parameters 🤯
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

In [11]:
# Compiling the model
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])

In [14]:
# Training the model
history = model.fit_generator(
    train_generator,
    epochs=15,
)

/var/folders/xp/g_y_0gnd0v37gfx63299g9_00000gn/T/ipykernel_38681/2970098988.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/15
33/33 [==============================] - 6s 139ms/step - loss: 0.4605 - accuracy: 0.8218
Epoch 2/15
33/33 [==============================] - 4s 130ms/step - loss: 0.2037 - accuracy: 0.9231
Epoch 3/15
33/33 [==============================] - 4s 130ms/step - loss: 0.1163 - accuracy: 0.9581
Epoch 4/15
33/33 [==============================] - 4s 130ms/step - loss: 0.2583 - accuracy: 0.9698
Epoch 5/15
33/33 [==============================] - 4s 133ms/step - loss: 0.1104 - accuracy: 0.9718
Epoch 6/15
33/33 [==============================] - 4s 130ms/step - loss: 0.7680 - accuracy: 0.9776
Epoch 7/15
33/33 [==============================] - 4s 129ms/step - loss: 0.0267 - accuracy: 0.9893
Epoch 8/15
33/33 [==============================] - 4s 130ms/step - loss: 9.5717e-04 - accuracy: 1.0000
Epoch 9/15
33/33 [==============================] - 4s 131ms/step - loss: 9.4963e-05 - accuracy: 1.0000
Epoch 10/15
33/33 [==============================] - 4s 130ms/step - loss: 0.2038 - accuracy